In [45]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sn


ModuleNotFoundError: No module named 'seaborn'

In [59]:
#Loading the dataset  
df = pd.read_csv('/Users/ahtiholli/Downloads/data/europarl.csv', header = None)
df.rename(columns={0: 'Language'}, inplace=True)
df.rename(columns={1: 'Text'}, inplace=True)
df.head()


,Language,Text
0,bg,Състав на Парламента: вж. протоколи
1,bg,Одобряване на протокола от предишното заседани...
2,bg,Състав на Парламента: вж. протоколи
3,bg,Проверка на пълномощията: вж. протоколи
4,bg,Внасяне на документи: вж. протоколи


In [60]:
#The dataset is sorted by language, thus we need to randomize the indexes it for optimizing learning
reindexed_df = df.sample(frac=1).reset_index(drop=True)
reindexed_df.head()
reindexed_df.count()



Language    832305
Text        832274
dtype: int64

In [61]:
#Separating the variables and then converting the categorial language value into a numerical value

reindexed_df = df.sample(frac=0.01, random_state=49) 
reindexed_df.count()
y = reindexed_df['Language']
X = reindexed_df['Text']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)



In [62]:
#Now we'll preprocess the text and remove any symbols or numbers and also transform the text into lower case
import re
list_of_sentences = []

for sentence in X:
    
    try:
        sentence = re.sub(r'[!@#$(),"%^*?:.;~`0-9]', '', sentence)
        sentence = re.sub(r'[[]]', '', sentence)
        sentence = sentence.lower()
        list_of_sentences.append(sentence)
        
    except TypeError:
        continue
        
        


In [64]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

#We'll apply the count vectorizer in chunks because of the large sample size and without the chunking the kernel kept crashing
cv = CountVectorizer()

cv.fit(list_of_sentences)

chunk_size = 500


X_final = None

for i in range(0, len(list_of_sentences), chunk_size):
    chunk_sentences = list_of_sentences[i:i + chunk_size]

    X_chunk = cv.transform(chunk_sentences).toarray()

    if X_final is None:
        X_final = X_chunk
    else:
        X_final = np.concatenate((X_final, X_chunk), axis=0)


print(X_final.shape)



(8323, 113908)


In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final,y, test_size = 0.2)

In [66]:
#We'll use Multinomial Native Bayes classification model for this instance for it's simplicity, quickness. For a more advanced solution neural networks should be considered as an option
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, Y_train)


MultinomialNB()

In [67]:
y_pred = model.predict(X_test)

In [68]:
from sklearn.metrics import accuracy_score, 
ac = accuracy_score(Y_test, y_pred)

print("Accuracy is :",ac)
#The accuracy is roughly 98%

Accuracy is : 0.9843843843843844


In [69]:
# Let's create a function to test the model
def predict(text):
     x = cv.transform([text]).toarray() 
     lang = model.predict(x) 
     lang = le.inverse_transform(lang) 
     print("The language is in",lang[0])
    
    

In [70]:
#English
predict('Machine learning (ML) is an umbrella term for solving problems for which development of algorithms by human programmers would be cost-prohibitive, and instead the problems are solved by helping machines discover their own algorithms')

The language is in en


In [71]:
#Finnish
predict('Koneoppiminen (ML) on sateenvarjotermi sellaisten ongelmien ratkaisemiselle, joihin algoritmien kehittäminen ihmisohjelmoijien toimesta olisi liian kallista, ja sen sijaan ongelmat ratkaistaan auttamalla koneita "löytämään" omat algoritminsa.')

The language is in fi


In [72]:
#Spanish
predict('El aprendizaje automático (AM) es un término genérico para resolver problemas para los que el desarrollo de algoritmos por parte de programadores humanos resultaría prohibitivo y, en su lugar, los problemas se resuelven ayudando a las máquinas a "descubrir" sus "propios" algoritmos.')

The language is in es


In [73]:
#French
predict("L'apprentissage automatique est un terme générique qui désigne la résolution de problèmes pour lesquels le développement d'algorithmes par des programmeurs humains serait prohibitif, et qui sont résolus en aidant les machines à découvrir leurs propres algorithmes.")

The language is in fr


In [74]:
#Portuguese
predict('A aprendizagem automática (AM) é um termo genérico para a resolução de problemas para os quais o desenvolvimento de algoritmos por programadores humanos teria custos proibitivos e, em vez disso, os problemas são resolvidos ajudando as máquinas a "descobrir" os seus "próprios" algoritmos')

The language is in pt


In [75]:
#Swedish
predict('Maskininlärning (ML) är ett samlingsnamn för problemlösning där det skulle vara för dyrt för mänskliga programmerare att utveckla algoritmer, och där problemen istället löses genom att hjälpa maskiner att "upptäcka" sina "egna" algoritmer')

The language is in sv
